In [59]:
"""
Trabalho computacional sobre algoritmos evolutivos para solução de sistemas de equações não lineares 
"""

'\nTrabalho computacional sobre algoritmos evolutivos para solução de sistemas de equações não lineares \n'

In [60]:
"""
Sistema de equações não lineares
"""
import numpy as np
import random
# evolution strategy (mu, lambda) of the ackley objective function
from numpy.random import randn
from numpy.random import rand
from numpy.random import seed



class EquationSystem:
    def __init__(self, equations):
        self.equations = equations

    def evaluate(self, x):
        return [np.abs(equation(x)) for equation in self.equations]


p1 = EquationSystem([
    lambda x: 0.8*(x[0]**2 + x[0] -1)*x[2] + 0.12*x[0]**2 + 2.16*x[0] - 0.12,
    lambda x: (1 + x[0]**2)*x[3] + 0.4*x[0]**2 - 1.6*x[0] - 0.4,
    lambda x: (1 + x[0]**2)*x[4] + x[0]**2 - 1,
    lambda x: (1 + x[0]**2)*x[5] + 0.8*(x[0]**2 + x[0] -1),
    lambda x: x[4]*x[6] - 0.02*x[5] - x[4] - x[2]*x[3] - 0.16*x[0],
    lambda x: x[6]**2 - 2*x[3]*x[6] + x[5]**2 + x[3]**2 - x[1]**2,
    lambda x: x[7] - x[1]*x[2],
    lambda x: 0.0476*x[2]*x[7]**12 + x[2] - 2.104
])

# p1 bounds
p1_lb = [-3, -1, -2, -1, -1, -0.5, -1.5, -1.5]
p1_ub = [1, 1, 2, 1, 1, 0.5, 1.5, 1.5]

In [61]:
p2 = EquationSystem([
    lambda x: x[4] + x[3] - 1.803,
    lambda x: (x[1] + x[2])*x[4] + 6.19116*x[3] - 1.803*(1.497 + 0.035),
    lambda x: x[5] + x[3] -0.328,
    lambda x: 0.28801*x[5] - x[1]*x[2]*x[4],
    lambda x: (-6.19116*x[0] + x[0]*x[2] + x[1]*x[4] - x[2]*x[4])*x[5] + x[0]*x[2]*x[4],
    lambda x: 1.571*x[6] + x[3] -1.803,
    lambda x: x[7] - 0.000856*x[6]**2,
    lambda x: (x[4] -x[0])*x[8] - x[0]*x[4],
    lambda x: x[8] - 377*x[1]*x[7]
])

# p2 bounds
p2_lb = [-0.5, -1, -1, -1, 1, -1, -1, 0, -1]
p2_ub = [0.5, 1, 1, 1, 2, 1, 1, 1, 1]

In [62]:
# Real-Coded Genetic Algorithm

# seed the pseudorandom number generator
seed(1)

def generate_individual(upper_bound: list, lower_bound: list) -> np.array:
    return np.array(
        [
            random.uniform(lower_bound[i], upper_bound[i])
            for i in range(len(upper_bound))
        ]
    )


def generate_population(upper_bound: list, lower_bound: list, pop_size) -> list:
    return [generate_individual(upper_bound, lower_bound) for _ in range(pop_size)]


# evaluate Using the penalty method
def evaluate_penalty(individual, equation_system):
    result = equation_system.evaluate(individual)
    penalty_sum = 0
    for r in result:
        penalty_sum += max(0, r) ** 2
    return penalty_sum

"""
 Um exemplo específico para o caso de codificação em ponto
flutuante é o chamado crossover aritmético (MICHALEWICZ, 1996). Este operador é
definido como uma fusão de dois vetores (cromossomos): se x1 e x2 são dois indivíduos
selecionados para crossover, os dois filhos resultantes serão x_1^line = alpha*x_1 + (1 - alpha)*x_2 e
x_2^line = (1 - alpha)*x_1 + alpha*x_2, sendo alpha um número aleatório pertencente ao intervalo [0, 1]. 
"""

def arithmetical_crossover(parent1, parent2, alpha):
    parent1, _ = parent1
    parent2, _ = parent2
    offspring1 = alpha * parent1 + (1 - alpha) * parent2
    offspring2 = (1 - alpha) * parent1 + alpha * parent2
    return (offspring1, None), (offspring2, None)

"""
o caso de problemas com codificação em ponto flutuante, o operador de mutação
mais popular é a mutação gaussiana (MICHALEWICZ & S CHOENAUER , 1996), modificando
todos os componentes de um cromossomo x = [x1 … xn] na forma:
x^line = x + N(0, sigma)

Sendo N(0, sigma) um vetor de variáveis aleatórias independentes, com distribuição normal,
média zero e desvio padrão sigma
"""

def gaussian_mutation(individual, sigma):
    ind, fitness = individual
    return ind + np.random.normal(0, sigma, len(ind)), fitness


def roulette_wheel_selection(evaluated_population):
    total_fitness = 0
    for individual, fitness in evaluated_population:
        total_fitness += 1/fitness
    r = random.uniform(0, total_fitness)
    acc = 0
    for individual, fitness in evaluated_population:
        acc += 1/fitness
        if acc >= r:
            return individual, fitness
    return evaluated_population[-1]


def real_coded_genetic_algorithm(equation_system, lower_bound, upper_bound, pop_size, max_gen, pc, pm, alpha, sigma, elitisim=True):
    """
    Real-Coded Genetic Algorithm
    :param equation_system: EquationSystem
    :param lower_bound: list
    :param upper_bound: list
    :param pop_size: int
    :param max_gen: int
    :param pc: float - Crossover probability
    :param pm: float - Mutation probability
    :param alpha: float - Crossover parameter
    :param sigma: float - Mutation parameter
    """
    # pop_size must be even
    if pop_size % 2 != 0:
        raise ValueError("pop_size must be even")

    population = generate_population(upper_bound, lower_bound, pop_size)
    population = [(individual, evaluate_penalty(individual, equation_system)) for individual in population]

    history = []
    for gen in range(max_gen):
        
        print(f"Generation {gen+1}", end="\r")
        # Evaluate
        evaluated_population = []
        # print(population)
        for individual, _ in population:
            fitness = evaluate_penalty(individual, equation_system)
            evaluated_population.append((individual, fitness))
        evaluated_population = sorted(evaluated_population, key=lambda x: x[1])

        # Keep 1 elite individual
        if elitisim:
            elite = evaluated_population[0]
        else:
            elite = None
        
        # Selection
        selected_population = []
        for _ in range(pop_size):
            selected_population.append(roulette_wheel_selection(evaluated_population))
        # Crossover
        offspring_population = []
        for i in range(0, pop_size, 2):
            if random.random() < pc:
                # print(selected_population[i], selected_population[i+1])
                offspring1, offspring2 = arithmetical_crossover(selected_population[i], selected_population[i+1], alpha)
                offspring_population.append(offspring1)
                offspring_population.append(offspring2)
            else:
                offspring_population.append(selected_population[i])
                offspring_population.append(selected_population[i+1])

        # Mutation
        mutated_population = []
        for individual in offspring_population:
            if random.random() < pm:
                mutated_population.append(gaussian_mutation(individual, sigma))
            else:
                mutated_population.append(individual)

        # Replace population
        population = mutated_population
        if elite:
            population[0] = elite
        
        best_fit = evaluated_population[0][1]
        history.append(best_fit)

    return evaluated_population[0], history


In [63]:
# run the genetic algorithm 30 times and collect the results
results = []
for _ in range(30):
    result, history = real_coded_genetic_algorithm(p1, p1_lb, p1_ub, 100, 2000, 0.8, 0.1, 0.5, 0.1, elitisim=True)
    best_individual, best_fitness = result
    results.append((result, history))

# statistical analysis
# taking the best, worst, mean, standard deviation and median of the results (using just the fitness values)
fitness_scores = [result[0][1] for result in results]



best = np.min(fitness_scores)
worst = np.max(fitness_scores)
mean = np.mean(fitness_scores)
std_dev = np.std(fitness_scores)
median = np.median(fitness_scores)

import pandas as pd

# create a dataframe to display the results
stats_df = pd.DataFrame({
    'Statistic': ['Best', 'Worst', 'Mean', 'Standard Deviation', 'Median'],
    'Value': [best, worst, mean, std_dev, median]
})

# import ace_tools as tools; tools.display_dataframe_to_user(name="GA Execution Statistics", dataframe=stats_df)


In [64]:
print(stats_df)

            Statistic     Value
0                Best  0.015884
1               Worst  1.057040
2                Mean  0.220946
3  Standard Deviation  0.335020
4              Median  0.049404


In [65]:
best_index = fitness_scores.index(best)
best_history = results[best_index][1]
print(best_history)

[3.92122700658094, 2.4134483201864647, 2.3659980639868747, 2.0080814980188766, 1.948752528931693, 1.948752528931693, 1.948752528931693, 1.948752528931693, 1.948752528931693, 1.948752528931693, 1.948752528931693, 1.8929481097127643, 1.8929481097127643, 1.8929481097127643, 1.6467190463555723, 1.6467190463555723, 1.6467190463555723, 1.6467190463555723, 1.6467190463555723, 1.6467190463555723, 1.6467190463555723, 1.6467190463555723, 1.5635121295064187, 1.5635121295064187, 1.5635121295064187, 1.5635121295064187, 1.5635121295064187, 1.5635121295064187, 1.5635121295064187, 1.5635121295064187, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.5498764611536167, 1.4263031188070125, 1.4263031188070125, 1.426303

In [66]:
# não tá tudo dando 0 ainda, tem q melhorar o algoritmo
p1.evaluate(best_individual)

[0.0979975098612127,
 0.16958325257280915,
 0.07612691687365047,
 0.030541877940946538,
 0.17805010519588593,
 0.040102223783184066,
 0.054095984788413265,
 0.01833838503028229]

In [67]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y=best_history, mode='lines'))
fig.show()

In [68]:

# penalty evaluation function
def evaluate_penalty(individual, equation_system):
    result = equation_system.evaluate(individual)
    penalty_sum = 0
    for r in result:
        penalty_sum += max(0, np.abs(r)) ** 2
    return penalty_sum

# check if a point is within the bounds of the search
def in_bounds(point, bounds):
    # enumerate all dimensions of the point
    for d in range(len(bounds)):
        # check if out of bounds for this dimension
        if point[d] < bounds[d, 0] or point[d] > bounds[d, 1]:
            return False
    return True

# evolution strategy (mu, lambda) algorithm
def es_comma(objective, bounds, n_iter, step_size, mu, lam):
    best, best_eval = None, 1e+10
    # calculate the number of children per parent
    n_children = int(lam / mu)
    # initial population
    population = list()
    history = []
    for _ in range(lam):
        candidate = None
        while candidate is None or not in_bounds(candidate, bounds):
            candidate = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
        population.append(candidate)
    # perform the search
    for epoch in range(n_iter):
        # evaluate fitness for the population
        scores = [objective(c) for c in population]
        # rank scores in ascending order
        ranks = np.argsort(np.argsort(scores))
        # select the indexes for the top mu ranked solutions
        selected = [i for i,_ in enumerate(ranks) if ranks[i] < mu]
        # create children from parents
        children = list()
        for i in selected:
            # check if this parent is the best solution ever seen
            if scores[i] < best_eval:
                best, best_eval = population[i], scores[i]
                history.append(best_eval)
                print('Epoch %d, Best: f(%s) = %.5f' % (epoch, best, best_eval))
            # create children for parent
            for _ in range(n_children):
                child = None
                while child is None or not in_bounds(child, bounds):
                    child = population[i] + randn(len(bounds)) * step_size
                children.append(child)
        # replace population with children
        population = children
    return [best, best_eval, history]


In [69]:

# define range for input
p1_lb = [-3, -1, -2, -1, -1, -0.5, -1.5, -1.5]
p1_ub = [1, 1, 2, 1, 1, 0.5, 1.5, 1.5]
# bounds = (min, max) for each variable
bounds = np.asarray([[p1_lb[i], p1_ub[i]] for i in range(len(p1_lb))])

print(bounds)
# define the total iterations
n_iter = 5000

# define the maximum step size
step_size = 0.15

# number of parents selected
mu = 20

# the number of children generated by parents
lam = 100

for _ in range(30):
    best, score, history = es_comma(lambda x: evaluate_penalty(x, p1), bounds, n_iter, step_size, mu, lam)
    # best_individual, best_fitness = result
    results.append(((best, score), history))

# perform the evolution strategy (mu, lambda) search
print('Done!')
print('f(%s) = %f' % (best, score))

# p1.evaluate(best)

# statistical analysis
# taking the best, worst, mean, standard deviation and median of the results (using just the fitness values)
fitness_scores = [result[0][1] for result in results]



best = np.min(fitness_scores)
worst = np.max(fitness_scores)
mean = np.mean(fitness_scores)
std_dev = np.std(fitness_scores)
median = np.median(fitness_scores)

# create a dataframe to display the results
stats_df = pd.DataFrame({
    'Statistic': ['Best', 'Worst', 'Mean', 'Standard Deviation', 'Median'],
    'Value': [best, worst, mean, std_dev, median]
})

best_index = fitness_scores.index(best)
best_history = results[best_index][1]
print(best_history)

# import ace_tools as tools; tools.display_dataframe_to_user(name="GA Execution Statistics", dataframe=stats_df)

[[-3.   1. ]
 [-1.   1. ]
 [-2.   2. ]
 [-1.   1. ]
 [-1.   1. ]
 [-0.5  0.5]
 [-1.5  1.5]
 [-1.5  1.5]]
Epoch 0, Best: f([-0.45364998  0.36035477  1.7544067   0.30583264  0.28320377  0.08295107
 -0.21187912  0.89393899]) = 10.45624
Epoch 0, Best: f([ 0.48461042  0.7353837   1.7658629   0.36132311 -0.3458657  -0.34758801
  0.57038925 -0.56320737]) = 6.60010
Epoch 0, Best: f([ 0.03191732 -0.78737049  0.70597238  0.74276398  0.42523324 -0.13668822
  1.2900229  -0.02783427]) = 4.08708
Epoch 0, Best: f([ 0.27468664 -0.01559846  0.88311171  0.18849253  0.64025857  0.22579985
 -0.15530095  1.20184707]) = 3.62263
Epoch 1, Best: f([ 0.33157601 -0.6470673   0.64817806  0.72785953  0.38671497 -0.34925336
  1.20711109 -0.3089137 ]) = 3.34486
Epoch 1, Best: f([ 0.39908034 -0.60195613  0.73040161  0.63059953  0.43786853 -0.03774927
  1.41707578 -0.150906  ]) = 2.73400
Epoch 2, Best: f([ 0.2903014  -0.65853584  0.70073573  0.5384696   0.4653965   0.06061631
  1.3085834  -0.19783052]) = 2.59103
Epoch

In [70]:
fig = go.Figure()

fig.add_trace(go.Scatter(y=best_history, mode='lines'))
fig.show()

In [71]:
for _ in range(30):
    result, history = real_coded_genetic_algorithm(p2, p2_lb, p2_ub, 100, 2000, 0.8, 0.1, 0.5, 0.1, elitisim=True)
    best_individual, best_fitness = result
    results.append((result, history))

fitness_scores = [result[0][1] for result in results]



best = np.min(fitness_scores)
worst = np.max(fitness_scores)
mean = np.mean(fitness_scores)
std_dev = np.std(fitness_scores)
median = np.median(fitness_scores)

import pandas as pd

# create a dataframe to display the results
stats_df = pd.DataFrame({
    'Statistic': ['Best', 'Worst', 'Mean', 'Standard Deviation', 'Median'],
    'Value': [best, worst, mean, std_dev, median]
})

best_index = fitness_scores.index(best)
best_history = results[best_index][1]
print(best_history)

fig = go.Figure()

fig.add_trace(go.Scatter(y=best_history, mode='lines'))
fig.show()

[12.877421152919474, 2.620510243442051, 1.6264506556275757, 1.469646451358809, 0.8871434394624388, 0.8871434394624388, 0.8871434394624388, 0.8871434394624388, 0.8871434394624388, 0.8871434394624388, 0.8871434394624388, 0.8871434394624388, 0.8871434394624388, 0.8871434394624388, 0.8871434394624388, 0.8871434394624388, 0.8871434394624388, 0.8332411454774962, 0.8332411454774962, 0.8173732682520901, 0.7944739945549472, 0.7211887672227473, 0.7211887672227473, 0.7211887672227473, 0.7211887672227473, 0.7211887672227473, 0.7211887672227473, 0.7211887672227473, 0.7211887672227473, 0.5460581175714176, 0.5460581175714176, 0.5460581175714176, 0.5460581175714176, 0.5460581175714176, 0.5460581175714176, 0.41229140061508557, 0.41229140061508557, 0.41229140061508557, 0.41229140061508557, 0.41229140061508557, 0.41229140061508557, 0.41229140061508557, 0.41229140061508557, 0.41229140061508557, 0.41229140061508557, 0.41229140061508557, 0.41229140061508557, 0.41229140061508557, 0.41229140061508557, 0.41229

In [1]:
# bounds = (min, max) for each variable
bounds = np.asarray([[p2_lb[i], p2_ub[i]] for i in range(len(p2_lb))])

print(bounds)
# define the total iterations
n_iter = 5000

# define the maximum step size
step_size = 0.15

# number of parents selected
mu = 20

# the number of children generated by parents
lam = 100

# perform the evolution strategy (mu, lambda) search
for _ in range(30):
    best, score, history = es_comma(lambda x: evaluate_penalty(x, p1), bounds, n_iter, step_size, mu, lam)
    # best_individual, best_fitness = result
    results.append(((best, score), history))

# perform the evolution strategy (mu, lambda) search
print('Done!')
print('f(%s) = %f' % (best, score))

# p1.evaluate(best)

# statistical analysis
# taking the best, worst, mean, standard deviation and median of the results (using just the fitness values)
fitness_scores = [result[0][1] for result in results]



best = np.min(fitness_scores)
worst = np.max(fitness_scores)
mean = np.mean(fitness_scores)
std_dev = np.std(fitness_scores)
median = np.median(fitness_scores)

# create a dataframe to display the results
stats_df = pd.DataFrame({
    'Statistic': ['Best', 'Worst', 'Mean', 'Standard Deviation', 'Median'],
    'Value': [best, worst, mean, std_dev, median]
})

best_index = fitness_scores.index(best)
best_history = results[best_index][1]
print(best_history)


import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y=best_history, mode='lines'))
fig.show()

NameError: name 'np' is not defined